In [1]:
import matplotlib.pyplot as plt
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import pickle
import joblib


In [6]:
layer_width = [50, 50]
n_inputs = 4
n_outputs = 8
input_data = pd.read_excel('/content/salidas_simulador-normal.xlsx')
output_data = pd.read_excel('/content/entradas_simulador-normal.xlsx')

dataset = pd.concat([input_data, output_data], axis=1)


In [7]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()


In [8]:
def norm(x):
 return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
train_data_as_numpy = normed_train_data.values
test_data_as_numpy = normed_test_data.values
x_train = train_data_as_numpy[:, 0:n_inputs]
y_train = train_data_as_numpy[:, n_inputs:]
x_test = test_data_as_numpy[:, 0:n_inputs]
y_test = test_data_as_numpy[:, n_inputs:]

def norm2(x):
 return (x - train_stats['mean'][0:n_inputs]) / train_stats['std'][0:n_inputs]


def norm3(x):
 return (x - train_stats['mean'][0:n_inputs]) / train_stats['std'][0:n_inputs]


In [9]:
for width in layer_width:
 model = Sequential()
 model.add(Dense(units=width, activation='sigmoid',
input_dim=n_inputs))
model.add(Dense(units=width, activation='sigmoid'))
 # model.add(Dense(units=width, activation='sigmoid'))
model.add(Dense(units=n_outputs, activation='sigmoid'))
model.compile(loss='mean_squared_error',
optimizer='Nadam', metrics=['mean_squared_error'])
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
min_delta=0, patience=2)


In [10]:
history = model.fit(x_train, y_train, validation_split=0.2,
epochs=50000, callbacks=[early_stop, keras.callbacks.TensorBoard()],
verbose=1)

Epoch 1/50000
80/80 [==============================] - 6s 6ms/step - loss: 1.1166 - mean_squared_error: 1.1166 - val_loss: 1.0034 - val_mean_squared_error: 1.0034
Epoch 2/50000
80/80 [==============================] - 0s 5ms/step - loss: 1.0093 - mean_squared_error: 1.0093 - val_loss: 0.9860 - val_mean_squared_error: 0.9860
Epoch 3/50000
80/80 [==============================] - 0s 5ms/step - loss: 0.9995 - mean_squared_error: 0.9995 - val_loss: 0.9791 - val_mean_squared_error: 0.9791
Epoch 4/50000
80/80 [==============================] - 0s 5ms/step - loss: 0.9906 - mean_squared_error: 0.9906 - val_loss: 0.9677 - val_mean_squared_error: 0.9677
Epoch 5/50000
80/80 [==============================] - 0s 5ms/step - loss: 0.9754 - mean_squared_error: 0.9754 - val_loss: 0.9512 - val_mean_squared_error: 0.9512
Epoch 6/50000
80/80 [==============================] - 0s 5ms/step - loss: 0.9574 - mean_squared_error: 0.9574 - val_loss: 0.9334 - val_mean_squared_error: 0.9334
Epoch 7/50000
80/80 [=

In [11]:
def plot_history(history):
 hist = pd.DataFrame(history.history)
 hist['epoch'] = history.epoch
 plt.figure()
 plt.xlabel('Epoch')
 plt.ylabel('Mean Square Error')
 plt.plot(hist['epoch'], hist['mean_squared_error'],
 label='Train Error')
 plt.plot(hist['epoch'], hist['val_mean_squared_error'],
 label='Val Error')
 plt.ylim([0, 1])
 plt.legend()
 plt.show()

In [12]:
#plot_history(history)
 # hist = pd.DataFrame(history.history)
 # with open('history.txt', 'wb') as file:
 # pickle.dump(history.history, file)
loss, mse = model.evaluate(x_test, y_test, verbose=0)
outputs = model.predict(x_test, verbose=0)
correlations = np.zeros(outputs.shape[1])
for i in range(len(correlations)):
  correlations[i] = np.corrcoef(y_test[:, i], outputs[:, i])[0, 1]
  print(normed_test_data.columns.values[26:])
  print(correlations)
 # model.save('calibration_nn.h5')
model

[]
[0.58972057 0.         0.         0.         0.         0.
 0.         0.        ]
[]
[0.58972057 0.45165294 0.         0.         0.         0.
 0.         0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.         0.         0.
 0.         0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.35413861 0.         0.
 0.         0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.35413861 0.86911037 0.
 0.         0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.35413861 0.86911037 0.87975344
 0.         0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.35413861 0.86911037 0.87975344
 0.88794661 0.        ]
[]
[0.58972057 0.45165294 0.30866618 0.35413861 0.86911037 0.87975344
 0.88794661 0.87123622]


In [13]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Calcular el MSE
mse = mean_squared_error(y_test, outputs)

# Calcular las correlaciones
correlations = np.zeros(outputs.shape[1])
for i in range(len(correlations)):
    correlations[i] = np.corrcoef(y_test[:, i], outputs[:, i])[0, 1]

# Imprimir el MSE
print("Mean Squared Error:", mse)

# Imprimir las correlaciones
for i, corr in enumerate(correlations):
    print("Correlation for variable", i+1, ":", corr)


Mean Squared Error: 0.7007259301095417
Correlation for variable 1 : 0.5897205718860047
Correlation for variable 2 : 0.4516529408137929
Correlation for variable 3 : 0.3086661805857168
Correlation for variable 4 : 0.3541386067156637
Correlation for variable 5 : 0.8691103706025285
Correlation for variable 6 : 0.8797534396934187
Correlation for variable 7 : 0.8879466103881364
Correlation for variable 8 : 0.8712362236180541


***NN predictions***

In [20]:
def prediction(new_inputs):
  normed_input_data = norm2(new_inputs)

  test_input_as_numpy = normed_input_data.values

  input_test = test_input_as_numpy[0:n_inputs]

  # Reshape de los datos de entrada
  input_test = input_test.reshape(1, -1)

  predictions_knn = model.predict(input_test)

  predictions_knn= predictions_knn.reshape(-1)

  desnormalized_predictions = (predictions_knn * train_stats['std'][n_inputs:]) + train_stats['mean'][n_inputs:]

  desnormalized_predictions_array = desnormalized_predictions.values
  return desnormalized_predictions_array

#print(prediction([46,16,246,178]))

***Random Forest***

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from scipy.stats import pearsonr  # Importar la función pearsonr

# Crear el modelo de Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=0)

# Entrenar el modelo con los datos de entrenamiento
model_rf.fit(x_train, y_train)

# Realizar predicciones en los datos de prueba
predictions_rf = model_rf.predict(x_test)

# Calcular el R2 para cada variable
r2_per_variable = r2_score(y_test, predictions_rf, multioutput='raw_values')

# Calcular el coeficiente de correlación de Pearson para cada variable
pearson_per_variable = [pearsonr(y_test[:, i], predictions_rf[:, i])[0] for i in range(predictions_rf.shape[1])]

# Imprimir el R2 y el coeficiente de correlación de Pearson para cada variable
for i, (r2, pearson) in enumerate(zip(r2_per_variable, pearson_per_variable)):
    print("Variable", i+1, "R2:", r2)
    print("Variable", i+1, "Pearson Correlation:", pearson)

mse = mean_squared_error(y_test, predictions_rf)

# Imprimir el MSE
print("Mean Squared Error:", mse)




Variable 1 R2: 0.3696994916647073
Variable 1 Pearson Correlation: 0.6134453595739623
Variable 2 R2: 0.20329271501229995
Variable 2 Pearson Correlation: 0.473048468105346
Variable 3 R2: 0.06925076629175342
Variable 3 Pearson Correlation: 0.31021715038273556
Variable 4 R2: 0.044380458287551994
Variable 4 Pearson Correlation: 0.30303814829518766
Variable 5 R2: 0.9330457856520143
Variable 5 Pearson Correlation: 0.9669730217905841
Variable 6 R2: 0.9132725666036018
Variable 6 Pearson Correlation: 0.9575945620322613
Variable 7 R2: 0.9221877921905017
Variable 7 Pearson Correlation: 0.9612901241929167
Variable 8 R2: 0.9106083585618756
Variable 8 Pearson Correlation: 0.956221049850359
Mean Squared Error: 0.44036765200389943


***KNN***

In [22]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
import joblib


# Crear el modelo KNN
model_knn = KNeighborsRegressor(n_neighbors=5)

# Entrenar el modelo con los datos de entrenamiento
model_knn.fit(x_train, y_train)
# Guardar el modelo en un archivo
joblib.dump(model_knn, 'modelo_knn.pkl')
# Realizar predicciones en los datos de prueba
predictions_knn = model_knn.predict(x_test)

# Calcular el R2 para cada variable
r2_per_variable = r2_score(y_test, predictions_knn, multioutput='raw_values')

# Imprimir el R2 para cada variable
for i, r2 in enumerate(r2_per_variable):
    print("R2 for variable", i+1, ":", r2)

mse = mean_squared_error(y_test, predictions_knn)

# Imprimir el MSE
print("Mean Squared Error:", mse)


R2 for variable 1 : 0.3460548725481357
R2 for variable 2 : 0.16706720663235397
R2 for variable 3 : -0.03485454784869013
R2 for variable 4 : -0.062376790773887025
R2 for variable 5 : 0.9212153890692787
R2 for variable 6 : 0.9072246332325437
R2 for variable 7 : 0.9198272296940564
R2 for variable 8 : 0.9112259151652996
Mean Squared Error: 0.4754223887767389


In [24]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from scipy.stats import pearsonr  # Importar la función pearsonr
import joblib

# Crear el modelo KNN
model_knn = KNeighborsRegressor(n_neighbors=5)

# Entrenar el modelo con los datos de entrenamiento
model_knn.fit(x_train, y_train)

# Realizar predicciones en los datos de prueba
predictions_knn = model_knn.predict(x_test)

# Calcular el R2 para cada variable
r2_per_variable = r2_score(y_test, predictions_knn, multioutput='raw_values')

# Calcular el coeficiente de correlación de Pearson para cada variable
pearson_per_variable = [pearsonr(y_test[:, i], predictions_knn[:, i])[0] for i in range(predictions_knn.shape[1])]

# Imprimir el R2 y el coeficiente de correlación de Pearson para cada variable
for i, (r2, pearson) in enumerate(zip(r2_per_variable, pearson_per_variable)):
    print("Variable", i+1, "R2:", r2)
    print("Variable", i+1, "Pearson Correlation:", pearson)

# Guardar el modelo en un archivo
joblib.dump(model_knn, 'modelo_knn.pkl')


Variable 1 R2: 0.3460548725481357
Variable 1 Pearson Correlation: 0.6020791175182237
Variable 2 R2: 0.16706720663235397
Variable 2 Pearson Correlation: 0.4558604183678373
Variable 3 R2: -0.03485454784869013
Variable 3 Pearson Correlation: 0.24173397907729996
Variable 4 R2: -0.062376790773887025
Variable 4 Pearson Correlation: 0.2561824587575172
Variable 5 R2: 0.9212153890692787
Variable 5 Pearson Correlation: 0.959812368015785
Variable 6 R2: 0.9072246332325437
Variable 6 Pearson Correlation: 0.9526123794855617
Variable 7 R2: 0.9198272296940564
Variable 7 Pearson Correlation: 0.9591277135781452
Variable 8 R2: 0.9112259151652996
Variable 8 Pearson Correlation: 0.9549742163917102


['modelo_knn.pkl']

***Other machine learning models***

In [26]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

# Lista para almacenar los modelos GBR
models_gbr = []

# Lista para almacenar los R2 por variable
r2_per_variable = []

# Entrenar un modelo GBR para cada variable de salida
for i in range(y_train.shape[1]):
    # Crear el modelo GBR
    model_gbr = GradientBoostingRegressor()

    # Entrenar el modelo con la variable i de y_train
    model_gbr.fit(x_train, y_train[:, i])

    # Agregar el modelo a la lista
    models_gbr.append(model_gbr)

    # Realizar predicciones en los datos de prueba para la variable i
    predictions_gbr = model_gbr.predict(x_test)

    # Calcular el R2 para la variable i
    r2 = r2_score(y_test[:, i], predictions_gbr)

    # Agregar el R2 a la lista
    r2_per_variable.append(r2)

# Imprimir el R2 para cada variable
for i, r2 in enumerate(r2_per_variable):
    print("R2 for variable", i+1, ":", r2)



R2 for variable 1 : 0.40743777571666917
R2 for variable 2 : 0.20678366177404495
R2 for variable 3 : 0.08872834948286934
R2 for variable 4 : 0.11472474267678545
R2 for variable 5 : 0.9230479805399907
R2 for variable 6 : 0.906039707739771
R2 for variable 7 : 0.9088908869059058
R2 for variable 8 : 0.9046578971678685


In [27]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# Crear una lista para almacenar los modelos SVR
models_svr = []

# Entrenar un modelo SVR por cada variable de salida
for i in range(y_train.shape[1]):
    model_svr = SVR()
    model_svr.fit(x_train, y_train[:, i])
    models_svr.append(model_svr)

# Realizar predicciones en los datos de prueba y calcular el R2 para cada variable
r2_per_variable = []
for i, model_svr in enumerate(models_svr):
    predictions_svr = model_svr.predict(x_test)
    r2 = r2_score(y_test[:, i], predictions_svr)
    r2_per_variable.append(r2)
    print("R2 for variable", i+1, ":", r2)


R2 for variable 1 : 0.40446813334239606
R2 for variable 2 : 0.20053709025703015
R2 for variable 3 : 0.011523694912924709
R2 for variable 4 : 0.02799323144502508
R2 for variable 5 : 0.928628187434112
R2 for variable 6 : 0.9100772161176526
R2 for variable 7 : 0.9339200662582293
R2 for variable 8 : 0.9117609512201047


***Predicciones del modelo KNN- Regressor (For genetic algorithm)***

In [ ]:
import numpy as np
import joblib

# Cargar el modelo desde el archivo
loaded_model = joblib.load('modelo_knn.pkl')

def prediction(new_inputs):
  normed_input_data = norm2(new_inputs)

  test_input_as_numpy = normed_input_data.values

  input_test = test_input_as_numpy[0:n_inputs]

  # Reshape de los datos de entrada
  input_test = input_test.reshape(1, -1)

  predictions_knn = model_knn.predict(input_test)

  predictions_knn= predictions_knn.reshape(-1)

  desnormalized_predictions = (predictions_knn * train_stats['std'][n_inputs:]) + train_stats['mean'][n_inputs:]

  desnormalized_predictions_array = desnormalized_predictions.values
  return desnormalized_predictions_array

print(prediction([75, 232, 51, 185, 0.33138852041968037, 0.26927802115768673, 0.17396830691811851, 0.2253651515045144]))

[170.6 147.2  99.6 141.2]


***Algoritmo genetico 2.0***

In [ ]:
import random
import numpy as np
from sklearn.metrics import mean_squared_error
def generar_conjunto_probabilidades():
    conjunto = np.random.dirichlet(np.ones(4))
    while any(prob <= 0.1 for prob in conjunto):
        conjunto = np.random.dirichlet(np.ones(4))
    return conjunto

# Función de evaluación de aptitud
def aptitud(cromosoma):
    # Obtener las combinaciones de volúmenes de entrada y probabilidades de ruta
    volumenes_entrada = cromosoma[:4]
    probabilidades_ruta = cromosoma[4:]
    entrada_modelo=np.concatenate([volumenes_entrada, probabilidades_ruta])
    # Realizar predicciones con el modelo de machine learning

    predicciones = prediction(entrada_modelo)

    # Calcular el error cuadrático medio entre las predicciones y la salida deseada
    salida_deseada = np.array([200, 120, 95, 340])
    error = mean_squared_error(salida_deseada, predicciones)

    # El objetivo es minimizar el error, por lo que se devuelve el inverso del error
    aptitud = 1 / (1 + error)
    return aptitud

# Función de mutación
def mutar(cromosoma):
    indice = random.randint(0, len(cromosoma) - 1)
    nuevo_valor = random.randint(0, 1000) if indice < 4 else random.uniform(0.1, 1)
    cromosoma[indice] = nuevo_valor

    # Ajustar las probabilidades de ruta para que sumen 1
    if indice >= 4:
        suma_probabilidades = sum(cromosoma[4:])
        cromosoma[4:] = [p / suma_probabilidades for p in cromosoma[4:]]

    return cromosoma

# Parámetros del algoritmo genético
tamaño_población = 100  # Tamaño de la población
probabilidad_mutación = 0.2  # Probabilidad de mutación
num_generaciones = 500  # Número de generaciones

# Crear la población inicial
población = []
for _ in range(tamaño_población):
    volumenes_entrada = [random.randint(0, 1000) for _ in range(4)]
    probabilidades_ruta = generar_conjunto_probabilidades()
    cromosoma = volumenes_entrada + probabilidades_ruta.tolist()
    población.append(cromosoma)



# Ciclo de evolución
for generacion in range(num_generaciones):
    # Evaluar la aptitud de la población
    aptitudes = [aptitud(cromosoma) for cromosoma in población]

    # Seleccionar los mejores cromosomas para la reproducción
    mejores_cromosomas = sorted(zip(población, aptitudes), key=lambda x: x[1], reverse=True)
    mejores_cromosomas = [cromosoma for cromosoma, _ in mejores_cromosomas[:int(tamaño_población/2)]]

    # Reproducción y mutación para generar nueva población
    descendencia = []
    while len(descendencia) < tamaño_población:
        padre = random.choice(mejores_cromosomas)
        madre = random.choice(mejores_cromosomas)
        hijo = padre[:4] + madre[4:]
        if random.random() < probabilidad_mutación:
            hijo = mutar(hijo)
        descendencia.append(hijo)

    # Reemplazar la población anterior con la nueva descendencia
    población = descendencia

    # Imprimir el mejor cromosoma de la generación actual
    mejor_cromosoma = max(zip(población, aptitudes), key=lambda x: x[1])[0]
    print(f"Generación {generacion+1}: {mejor_cromosoma}")

# Imprimir el mejor cromosoma obtenido
mejor_cromosoma = max(zip(población, aptitudes), key=lambda x: x[1])[0]
print("Mejor cromosoma:", mejor_cromosoma)

Generación 1: [433, 600, 980, 848, 0.2718548013622028, 0.22796426091763608, 0.12086506215072423, 0.37931587556943674]
Generación 2: [901, 231, 384, 908, 0.24038964600326426, 0.11062937547568813, 0.2786899940509117, 0.37029098447013603]
Generación 3: [350, 89, 75, 414, 0.16726673534084957, 0.11335799163372053, 0.48603304893418686, 0.23334222409124294]
Generación 4: [389, 131, 954, 568, 0.1886175970164504, 0.1718904751962419, 0.11763020553820903, 0.5218617222490985]
Generación 5: [373, 295, 45, 299, 0.2718548013622028, 0.22796426091763608, 0.12086506215072423, 0.37931587556943674]
Generación 6: [25, 181, 199, 578, 0.28762226073740854, 0.1949238846558655, 0.12485728788552752, 0.3925965667211985]
Generación 7: [389, 131, 954, 568, 0.28515699148151563, 0.19325315215119318, 0.12378711052022484, 0.39780274584706626]
Generación 8: [25, 181, 199, 578, 0.2512550422727119, 0.10776461299007224, 0.1836951913630412, 0.45728515337417475]
Generación 9: [25, 181, 199, 578, 0.2512550422727119, 0.1077646